In [13]:
from striprtf.striprtf import rtf_to_text
import json

with open(r"C:\Users\91962\Downloads\DS_Assignment - internship\Screening Test - DS\algoparams_from_ui.json.rtf", "r", encoding="utf-8") as file:
    rtf_content = file.read()
    
text = rtf_to_text(rtf_content)
dic = json.loads(text)

In [14]:
dic

{'session_name': 'test',
 'session_description': 'test',
 'design_state_data': {'session_info': {'project_id': '1',
   'experiment_id': 'kkkk-11',
   'dataset': 'iris_modified.csv',
   'session_name': 'test',
   'session_description': 'test'},
  'target': {'prediction_type': 'Regression',
   'target': 'petal_width',
   'type': 'regression',
   'partitioning': True},
  'train': {'policy': 'Split the dataset',
   'time_variable': 'sepal_length',
   'sampling_method': 'No sampling(whole data)',
   'split': 'Randomly',
   'k_fold': False,
   'train_ratio': 0.7,
   'random_seed': 0},
  'metrics': {'optomize_model_hyperparameters_for': 'AUC',
   'optimize_threshold_for': 'F1 Score',
   'compute_lift_at': 0,
   'cost_matrix_gain_for_true_prediction_true_result': 1,
   'cost_matrix_gain_for_true_prediction_false_result': 0,
   'cost_matrix_gain_for_false_prediction_true_result': 0,
   'cost_matrix_gain_for_false_prediction_false_result': 0},
  'feature_handling': {'sepal_length': {'feature_nam

In [15]:
dic.keys()

dict_keys(['session_name', 'session_description', 'design_state_data'])

In [1]:
from striprtf.striprtf import rtf_to_text

import pandas as pd
import numpy as np
import json
import re
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet, SGDClassifier, SGDRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.calibration import CalibratedClassifierCV
import xgboost as xgb
import time

In [61]:
class preprocess_pipeline():
    def __init__(self, dic):
        self.hashed = []
        self.dic = dic
        
    def read_df(self):
        file = dic["design_state_data"]["session_info"]["dataset"]
        df = pd.read_csv(file)
        print("READ THE DATAFRAME")
        return df
        
    def data_selection(self):
        self.df = self.read_df()
        # target vars
        pred = self.dic["design_state_data"]["target"]["prediction_type"]
        self.target = self.dic["design_state_data"]["target"]["target"]
        type_ = self.dic["design_state_data"]["target"]["type"]
        partition = self.dic["design_state_data"]["target"]["partitioning"]

        # train vars
        train_columns = [x for x in self.df.columns if x != self.target]
        train_size = self.dic["design_state_data"]["train"]["train_ratio"]
        
        policy = self.dic["design_state_data"]["train"]["policy"]
        split1 = "random" in self.dic["design_state_data"]["train"]["policy"].lower()
        split2 = "strat" in self.dic["design_state_data"]["train"]["policy"].lower()
        randomness = self.dic["design_state_data"]["train"]["random_seed"]
        KFOLD = self.dic["design_state_data"]["train"]["k_fold"]
        
        # train test split
        if "split" in policy.lower():
            X_train, X_test, Y_train, Y_test = train_test_split(
                                                                self.df[train_columns], 
                                                                self.df[self.target], 
                                                                train_size = train_size, 
                                                                shuffle = split1,
                                                                #stratify = split2,
                                                                random_state = randomness 
                                                                )
            
            X_train, X_test, Y_train, Y_test = self.feature_handling(X_train, X_test, Y_train, Y_test)
            X_train, X_test = self.feature_generation(X_train, X_test)
            print("SPLIT IMPUTATION COMPLETE")
            return "normal_split", [X_train, X_test, Y_train, Y_test]
            
        if KFOLD:
            kf = KFold(n_splits=2, shuffle=True, random_state=42)
            folds = []
            # Generate splits
            for train_idx, test_idx in kf.split(df):
                X_train = df.loc[train_idx, train_columns]
                X_test = df.loc[test_idx, train_columns]
                Y_train = df.loc[train_idx, target]
                Y_test = df.loc[test_idx, target]
                
                X_train, X_test, Y_train, Y_test = self.feature_handling(X_train, X_test, Y_train, Y_test)
                X_train, X_test = self.feature_generation(X_train, X_test)
                folds.append([X_train, X_test, Y_train, Y_test])
                
            print("KFOLDED IMPUTATION COMPLETE")
            return "kfold", folds
    
    def feature_handling(self, X_train, X_test, Y_train, Y_test):
        # list features
        features = list(self.dic["design_state_data"]["feature_handling"].keys())
        # grab the feature dictionary part
        features_dic = self.dic["design_state_data"]["feature_handling"]
        #print(features_dic)
        for feature in features:
            if features_dic[feature]["is_selected"]:
                if features_dic[feature]["feature_variable_type"].lower() == "numerical":
                    imp_method = features_dic[feature]["feature_details"]["impute_with"].lower()
                    if imp_method == "custom":
                        imp_value = features_dic[feature]["feature_details"]["impute_value"]
                        if feature == self.target:
                            Y_train.fillna(imp_value,  inplace = True)
                            Y_test.fillna(imp_value,  inplace = True) 
                        else:
                            X_train.fillna({feature : imp_value}, inplace = True)
                            X_test.fillna({feature : imp_value},  inplace = True)
    
                    else:
                        if features_dic[feature]["feature_variable_type"].lower() == "numerical":
                            if "mean" in imp_method or "average" in imp_method:
                                strategy = "mean"
                            elif "median" in imp_method:
                                strategy = "median"
                            elif "mode" in imp_method:
                                strategy = "mode"
                            si = SimpleImputer(strategy = strategy)
    
                            if feature == self.target:
                                si.fit(Y_train)
                                Y_train = si.transform(Y_train)
                                Y_test = si.transform(Y_test)
                            else:
                                si.fit(X_train[feature].values.reshape(-1, 1))
                                X_train[feature] = si.transform(X_train[feature].values.reshape(-1, 1)).ravel()
                                X_test[feature] = si.transform(X_test[feature].values.reshape(-1, 1)).ravel()
                                
                else: # for text data
                    self.hashed.append(feature)
                    strategy = "most_frequent"
    
                    si = SimpleImputer(strategy = strategy)
    
                    if feature == self.target:
                        si.fit(Y_train.values.reshape(-1, 1))
                        Y_train = si.transform(Y_train.values.reshape(-1, 1)).ravel()
                        Y_test = si.transform(Y_test.values.reshape(-1, 1)).ravel()
                    else:
                        si.fit(X_train[feature].values.reshape(-1, 1))
                        X_train[feature] = si.transform(X_train[feature].values.reshape(-1, 1)).ravel()
                        X_test[feature] = si.transform(X_test[feature].values.reshape(-1, 1)).ravel()
                    """
                    not expecting the target variable to be hashed or tokenized so ignoring y_train, 
                    but if it is, we will use Label encoder instead
                    """
                    if feature == self.target:
                        e = LabelEncoder()
                        e.fit(Y_train)
                        Y_train = e.transform(Y_train)
                        Y_test = e.transform(Y_test)
                        
                    else:
                        # use hashing evctorizer
                        hash_columns = features_dic[feature]["feature_details"]["hash_columns"]
                        if hash_columns != 0: 
                            vectorizer = HashingVectorizer(n_features = hash_columns, stop_words = 'english', alternate_sign = False)
            
                            hashed_X_train = vectorizer.transform(X_train[feature].astype(str)).toarray()
                            hashed_X_test = vectorizer.transform(X_test[feature].astype(str)).toarray()
                            
                            # vectorizer returns dense array create columns out of that 
                            hashed_X_train_df = pd.DataFrame(hashed_X_train, index=X_train.index,
                                  columns=[f"hashed_{feature}_{i}" for i in range(hashed_X_train.shape[1])])
                            hashed_X_test_df = pd.DataFrame(hashed_X_test, index=X_test.index,
                                 columns=[f"hashed_{feature}_{i}" for i in range(hashed_X_test.shape[1])])

                            # join
                            X_train = pd.concat([X_train, hashed_X_train_df], axis=1)
                            X_test = pd.concat([X_test, hashed_X_test_df], axis=1)

                            # drop the string col
                            X_train.drop(feature, axis = 1, inplace = True)
                            X_test.drop(feature, axis = 1, inplace = True)
                            
                        else:
                            e = LabelEncoder()
                            e.fit(X_train[feature])
                            X_train[feature] = e.transform(X_train[feature].values.reshape(-1, 1)).flatten()
                            X_test[feature]= e.transform(X_test[feature].values.reshape(-1, 1)).flatten()
                            
        print("Feature handling complete")
        #display(X_test)
        return X_train, X_test, Y_train, Y_test  

    def interaction(self, interaction_list, interaction_type, X_train, X_test):
        """
        handles interaction and takes care of hashed array interactions if any 
        """
        functiondic = {
                       "linear" : lambda x, y : x + y,
                       "poly" : lambda x, y : x / y, 
                       "expl" : lambda x, y : x / y
                      }
        
        function = functiondic[interaction_type]
        for x, y in interaction_list:
            # skip when target is given for interaction, this causes dataleakage
            if y == self.target: continue
            if x == self.target: continue
            
            if x in self.hashed:
                cols = [p for p in X_train.columns if f"hashed_{x}" in p]
                for hashed_col in cols:
                    X_train[f"{hashed_col}_l_{y}"] = function(X_train[hashed_col], X_train[y])
                    X_test[f"{hashed_col}_l_{y}"] = function(X_test[hashed_col], X_test[y])

            elif y in self.hashed:
                cols = [p for p in X_train.columns if f"hashed_{y}" in p]
                for hashed_col in cols:
                    X_train[f"{x}_l_{hashed_col}"] = function(X_train[x], X_train[hashed_col])
                    X_test[f"{x}_l_{hashed_col}"] = function(X_test[x], X_test[hashed_col])
            
            # not expecting both to be hashed would be worst case scenario
            elif x in self.hashed and y in self.hashed:
                colx = [p for p in X_train.columns if f"hashed_{x}" in p]
                coly = [p for p in X_train.columns if f"hashed_{y}" in p]
                
                for hashed_x in colx:
                    for hashed_y in coly:
                        X_train[f"{hashed_x}_l_{hashed_y}"] = function(X_train[hashed_x], X_train[hashed_y])
                        X_test[f"{hashed_x}_l_{hashed_y}"] = function(X_test[hashed_x], X_test[hashed_y])
            else:
                X_train[f"{x}_l_{y}"] = function(X_train[x], X_train[y])
                X_test[f"{x}_l_{y}"] = function(X_test[x], X_test[y])
        print("INTERACTION COMPLETE")
        return X_train, X_test


    def feature_generation(self, X_train, X_test):
        dic = self.dic["design_state_data"]["feature_generation"]
        
        # add new linear features
        X_train, X_test = self.interaction(dic["linear_interactions"], "linear", X_train, X_test)
            
        # add polynomial features
        polynomial_interactions = [x.split("/") for x in dic["polynomial_interactions"]]
        X_train, X_test = self.interaction(polynomial_interactions, "poly", X_train, X_test)
        
        # add explicit pariwaise reln
        explicit_pairwise_interactions = [x.split("/") for x in dic["explicit_pairwise_interactions"]]
        X_train, X_test = self.interaction(explicit_pairwise_interactions, "expl", X_train, X_test)
        print("FEATURE GENERATION COMPLETE")
        return X_train, X_test

In [62]:
with open(r"C:\Users\91962\Downloads\DS_Assignment - internship\Screening Test - DS\algoparams_from_ui.json.rtf", "r", encoding="utf-8") as file:
    rtf_content = file.read()
    
text = rtf_to_text(rtf_content)
dic = json.loads(text)

k = preprocess_pipeline(dic)
u = k.data_selection()

READ THE DATAFRAME
Feature handling (45, 13)


,sepal_length,sepal_width,petal_length,hashed_species_0,hashed_species_1,hashed_species_2,hashed_species_3,hashed_species_4,hashed_species_5,hashed_species_6,hashed_species_7,hashed_species_8,hashed_species_9
105,7.6,3.0,6.6,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
106,4.9,2.5,4.5,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
107,7.3,2.9,6.3,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
108,6.7,2.5,5.8,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
109,7.2,3.6,6.1,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
110,6.5,3.2,5.1,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
111,6.4,2.7,5.3,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
112,6.8,3.0,5.5,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
113,5.7,2.5,5.0,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0
114,5.8,2.8,5.1,0.0,0.707107,0.0,0.0,0.0,0.0,0.707107,0.0,0.0,0.0


interaction (45, 14)
interaction (45, 14)
interaction (45, 15)
generation (45, 15)
SPLIT IMPUTATION COMPLETE


In [63]:
u[1][1].shape

(45, 15)

In [64]:
class feature_reduction_pipeline:
    def __init__(self, dic, X_train, X_test, target):
        self.dic = dic
        self.method = dic["feature_reduction_method"].lower()
        self.X_train = X_train.copy()
        self.X_test = X_test.copy()
        self.target = target
    
    def execute(self):
        if "correlation" in self.method:
            return self.correlation(0.4)
            
        elif "principal" in self.method:
            return self.pca(self.dic[self.method]["num_features_to_keep"])
            
        elif "tree" in self.method:
            depth = self.dic[self.method]["depth_of_trees"]
            numt = self.dic[self.method]["num_of_trees"]
            n = self.dic[self.method]["num_features_to_keep"]
            return self.tree(num_features_to_keep=n, num_trees=numt, depth=depth)
        
        else:
            return self.X_train, self.X_test
            
    def correlation(self, threshold=0.4):
        combined = pd.concat([self.X_train, self.X_test])
        corr_matrix = combined.corr().abs()
         # Keeps the values where the condition is True and replaces everything else with NaN.                  
        """                 |    # makes it triangle # creates a corelation matrix like array with ones
                            |           |                |
                            v           v                v                 """
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = []
        
        for column in upper.columns:
            if any(upper[column] >= threshold):
                to_drop.append(column)
                
        keep = [column for column in corr_matrix.columns if column not in to_drop]
        
        return self.X_train[keep], self.X_test[keep]
    
    def pca(self, n=3):
        combined = pd.concat([self.X_train, self.X_test])
        
        pca = PCA(n_components=n)
        pca.fit(combined)
        
        train_transformed = pca.transform(self.X_train)
        test_transformed = pca.transform(self.X_test)
        
        cols = [f'PC{i+1}' for i in range(n)]
        train_pca = pd.DataFrame(train_transformed, index = self.X_train.index, columns = cols)
        test_pca = pd.DataFrame(test_transformed, index = self.X_test.index, columns = cols)
        
        return train_pca, test_pca
    
    def tree(self, num_features_to_keep=10, num_trees=100, depth=6):
        
        if self.target in self.X_train.columns:
            X = self.X_train.drop(columns=[self.target])
            y = self.X_train[self.target]
        else:
            X = self.X_train
        
        rf = RandomForestClassifier(n_estimators = num_trees, max_depth = depth)
        rf.fit(X, y)
        
        # feature importances
        importances = rf.feature_importances_
        feature_names = X.columns.tolist()
        
        # sort features on importance
        features_with_importance = list(zip(feature_names, importances))
        sorted_features = sorted(features_with_importance, key=lambda x: x[1], reverse=True)
        
        # select top N features
        selected_features = [f[0] for f in sorted_features[:num_features_to_keep]]
        
        X_train_reduced = self.X_train[selected_features]
        X_test_reduced = self.X_test[selected_features]
        
        return X_train_reduced, X_test_reduced

In [65]:
local_dic = dic["design_state_data"]["feature_reduction"]
target = dic["design_state_data"]["target"]["target"]
dimred = feature_reduction_pipeline(local_dic, u[1][0], u[1][1], target)
k = dimred.execute()

In [66]:
k[0]

,sepal_length,sepal_width,hashed_species_0,hashed_species_2,hashed_species_3,hashed_species_4,hashed_species_5,hashed_species_6,hashed_species_8,hashed_species_9
0,5.1,3.5,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
1,4.9,3.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
2,4.7,3.2,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
3,4.6,3.1,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
4,5.0,3.6,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
100,6.3,3.3,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
101,5.8,2.7,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
102,7.1,3.0,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0
103,6.3,2.9,0.0,0.0,0.0,0.0,0.0,0.707107,0.0,0.0


In [67]:
class model_pipeline:
    def __init__(self, dic):
        self.dic = dic
        # get design state data part
        self.design_data = self.dic["design_state_data"]
        # get pred type
        self.prediction_type = self.design_data["target"]["prediction_type"]
        # get algo
        self.selected_algorithms = self._get_selected_algorithms()
        self.best_model = None
        self.best_score = float('-inf') if self.prediction_type == "Classification" else float('inf')
        self.best_params = None
        
    def _get_selected_algorithms(self):
        selected = []
        for algo_name, algo_config in self.design_data["algorithms"].items():
            if algo_config["is_selected"]:
                selected.append((algo_name, algo_config))
                #print(algo_name, algo_config)
        return selected
        
    def apply_sample_weights(self, X_train, Y_train):
        weight_strategy = self.design_data["weighting_stratergy"]
        if weight_strategy["weighting_stratergy_method"] == "Sample weights":
            weight_var = weight_strategy["weighting_stratergy_weight_variable"]
            
            if weight_var in X_train.columns:
                sample_weights = X_train[weight_var].values
                # remove weight variable from features
                X_train = X_train.drop(columns=[weight_var])
                return X_train, Y_train, sample_weights
                
        return X_train, Y_train, None
    
    def create_grid_params(self, algo_name, algo_config):
        if algo_name == "RandomForestClassifier" or algo_name == "RandomForestRegressor":
            return  {
                'n_estimators': list(range(algo_config["min_trees"], algo_config["max_trees"] + 1, 5)),
                'max_depth': list(range(algo_config["min_depth"], algo_config["max_depth"] + 1, 5)),
                'min_samples_leaf': list(range(algo_config["min_samples_per_leaf_min_value"], 
                                           algo_config["min_samples_per_leaf_max_value"] + 1, 5))
            }
            
        elif algo_name == "GBTClassifier" or algo_name == "GBTRegressor":
            return {
                'n_estimators': algo_config["num_of_BoostingStages"],
                'max_depth': list(range(algo_config["min_depth"], algo_config["max_depth"] + 1)),
                'learning_rate': [algo_config["min_stepsize"] + i * 0.05 for i in range(
                    int((algo_config["max_stepsize"] - algo_config["min_stepsize"]) / 0.05) + 1)],
                'subsample': [algo_config["min_subsample"] + i * 0.1 for i in range(
                    int((algo_config["max_subsample"] - algo_config["min_subsample"]) / 0.1) + 1)]
            }
            
        elif algo_name == "LinearRegression" or algo_name == "LogisticRegression":
            return {
                'max_iter': list(range(algo_config["min_iter"], algo_config["max_iter"] + 10, 10)),
                'tol': [0.0001, 0.001, 0.01],
                'C': [1.0 / (algo_config["min_regparam"] + i * 0.1) for i in range(
                    int((algo_config["max_regparam"] - algo_config["min_regparam"]) / 0.1) + 1)],  
                'l1_ratio': [algo_config["min_elasticnet"] + i * 0.1 for i in range(
                    int((algo_config["max_elasticnet"] - algo_config["min_elasticnet"]) / 0.1) + 1)] if "elasticnet" in algo_config else [0.5]
            }
            
        elif algo_name == "RidgeRegression":
            return {
                'max_iter': list(range(algo_config["min_iter"], algo_config["max_iter"] + 10, 10)),
                'alpha': [algo_config["min_regparam"] + i * 0.1 for i in range(
                    int((algo_config["max_regparam"] - algo_config["min_regparam"]) / 0.1) + 1)]
            }
            
        elif algo_name == "LassoRegression":
            return {
                'max_iter': list(range(algo_config["min_iter"], algo_config["max_iter"] + 10, 10)),
                'alpha': [algo_config["min_regparam"] + i * 0.1 for i in range(
                    int((algo_config["max_regparam"] - algo_config["min_regparam"]) / 0.1) + 1)]
            }
            
        elif algo_name == "ElasticNetRegression":
            return {
                'max_iter': list(range(algo_config["min_iter"], algo_config["max_iter"] + 10, 10)),
                'alpha': [algo_config["min_regparam"] + i * 0.1 for i in range(
                    int((algo_config["max_regparam"] - algo_config["min_regparam"]) / 0.1) + 1)],
                'l1_ratio': [algo_config["min_elasticnet"] + i * 0.1 for i in range(
                    int((algo_config["max_elasticnet"] - algo_config["min_elasticnet"]) / 0.1) + 1)]
            }
            
        elif algo_name == "xg_boost":
            return {
                'max_depth': list(range(algo_config["max_depth_of_tree"][0], algo_config["max_depth_of_tree"][1] + 1, 10)),
                'learning_rate': algo_config["learningRate"],
                'n_estimators': [10, 20, 50, 100],
                'gamma': algo_config["gamma"] if "gamma" in algo_config else [0],
                'reg_alpha': algo_config["l1_regularization"] if "l1_regularization" in algo_config else [0],
                'reg_lambda': algo_config["l2_regularization"] if "l2_regularization" in algo_config else [1],
                'subsample': algo_config["sub_sample"] if "sub_sample" in algo_config else [0.8],
                'colsample_bytree': algo_config["col_sample_by_tree"] if "col_sample_by_tree" in algo_config else [0.8]
            }
            
        elif algo_name == "DecisionTreeRegressor" or algo_name == "DecisionTreeClassifier":
            return {
                'max_depth': list(range(algo_config["min_depth"], algo_config["max_depth"] + 1)),
                'min_samples_leaf': algo_config["min_samples_per_leaf"],
                'criterion': ['gini' if algo_config.get("use_gini", False) else 'entropy'] if algo_name.endswith("Classifier") else ['squared_error', 'friedman_mse']
            }
            
        elif algo_name == "SVM":
            kernels = []
            if algo_config.get("linear_kernel", False): kernels.append('linear')
            if algo_config.get("polynomial_kernel", False): kernels.append('poly')
            if algo_config.get("sigmoid_kernel", False): kernels.append('sigmoid')
            if algo_config.get("rep_kernel", False): kernels.append('rbf')
            
            gamma_values = ['auto', 'scale']
            if algo_config.get("custom_gamma_values", False):
                gamma_values.extend([0.001, 0.01, 0.1, 1])
                
            return {
                'C': algo_config["c_value"],
                'kernel': kernels if kernels else ['rbf'],
                'gamma': gamma_values,
                'tol': [algo_config["tolerance"] * 0.001],
                'max_iter': [algo_config["max_iterations"] * 100]
            }
            
        elif algo_name == "SGD":
            losses = []
            if algo_config.get("use_logistics", False): losses.append('log_loss')
            if algo_config.get("use_modified_hubber_loss", False): losses.append('modified_huber')
            regularizations = []
            if algo_config.get("use_l1_regularization") == "on": regularizations.append('l1')
            if algo_config.get("use_l2_regularization") == "on": regularizations.append('l2')
            if algo_config.get("use_elastic_net_regularization", False): regularizations.append('elasticnet')
            
            return {
                'loss': losses if losses else ['log_loss'],
                'penalty': regularizations if regularizations else ['l2'],
                'alpha': algo_config["alpha_value"],
                'max_iter': [algo_config["max_iterations"] * 10] if "max_iterations" in algo_config else [1000],
                'tol': [algo_config["tolerance"] * 0.0001] if "tolerance" in algo_config else [0.0001]
            }
            
        elif algo_name == "KNN":
            return {
                'n_neighbors': algo_config["k_value"],
                'weights': ['uniform', 'distance'] if algo_config.get("distance_weighting", False) else ['uniform'],
                'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'] if algo_config.get("neighbour_finding_algorithm") == "Automatic" else [algo_config.get("neighbour_finding_algorithm").lower()],
                'p': [algo_config["p_value"]] if "p_value" in algo_config and algo_config["p_value"] > 0 else [2]
            }
            
        elif algo_name == "extra_random_trees":
            return {
                'n_estimators': [x for x in range(algo_config["num_of_trees"][0], algo_config["num_of_trees"][1] + 10, 10)],
                'max_depth': [x for x in range(algo_config["max_depth"][0], algo_config["max_depth"][1] + 1)],
                'min_samples_leaf': algo_config["min_samples_per_leaf"],
                'n_jobs': [algo_config["parallelism"]] if "parallelism" in algo_config else [1]
            }
            
        elif algo_name == "neural_network":
            return {
                'hidden_layer_sizes': [(algo_config["hidden_layer_sizes"][0],), (algo_config["hidden_layer_sizes"][1],)],
                'activation': ['relu', 'tanh', 'logistic'] if not algo_config.get("activation") else [algo_config.get("activation")],
                'alpha': [algo_config["alpha_value"]] if "alpha_value" in algo_config else [0.0001],
                'max_iter': [algo_config["max_iterations"]] if "max_iterations" in algo_config else [200],
                'tol': [algo_config["convergence_tolerance"]] if "convergence_tolerance" in algo_config else [0.0001],
                'learning_rate_init': [algo_config["initial_learning_rate"]] if "initial_learning_rate" in algo_config else [0.001],
                'solver': [algo_config["solver"]] if "solver" in algo_config else ['adam'],
                'batch_size': ['auto'] if algo_config.get("automatic_batching", True) else [200],
                'momentum': [algo_config["momentum"]] if "momentum" in algo_config else [0.9],
                'nesterovs_momentum': [algo_config["use_nesterov_momentum"]] if "use_nesterov_momentum" in algo_config else [True]
            }
            
        else:
            return {}
    
    def get_model(self, algo_name):

        if self.prediction_type.lower() == "classification":
            models = {
                "RandomForestClassifier": RandomForestClassifier(),
                "GBTClassifier": GradientBoostingClassifier(),
                "LogisticRegression": LogisticRegression(solver='saga'),
                "xg_boost": xgb.XGBClassifier(),
                "DecisionTreeClassifier": DecisionTreeClassifier(),
                "SVM": SVC(probability=True),
                "SGD": SGDClassifier(),
                "KNN": KNeighborsClassifier(),
                "extra_random_trees": RandomForestClassifier(bootstrap=False, random_state=42),
                "neural_network": MLPClassifier()
            }
            
        else:
            models = {
                "RandomForestRegressor": RandomForestRegressor(),
                "GBTRegressor": GradientBoostingRegressor(),
                "xg_boost": xgb.XGBRegressor(),
                "DecisionTreeRegressor": DecisionTreeRegressor(),
                "SGD": SGDRegressor(),
                "KNN": KNeighborsRegressor(),
                "extra_random_trees": RandomForestRegressor(bootstrap=False, random_state=42),
                "neural_network": MLPRegressor()
            }

        return models[algo_name]


    def train_and_evaluate(self, X_train, X_test, Y_train, Y_test):

        results = []
        
        X_train, Y_train, sample_weights = self.apply_sample_weights(X_train, Y_train)
        #print(self.selected_algorithms)
        
        for algo_name, algo_config in self.selected_algorithms:
            print(f"Training {algo_name}...")
            model_base = self.get_model(algo_name)
            print(model_base)
            params = self.create_grid_params(algo_name, algo_config)
            
            cv = KFold(n_splits=self.design_data["hyperparameters"].get("num_of_folds", 3), 
                       shuffle=self.design_data["hyperparameters"].get("shuffle_grid", True),
                       random_state=self.design_data["hyperparameters"].get("random_state", 42))
            
            if self.prediction_type.lower() == "classification":
                scoring = 'accuracy'
            else:
                scoring = 'neg_mean_squared_error'
            
            grid_search = GridSearchCV(
                estimator=model_base,
                param_grid=params,
                scoring=scoring,
                cv=cv,
                n_jobs=self.design_data["hyperparameters"].get("parallelism", -1),
                verbose=1
            )
            # to get the process time
            start_time = time.time()
            
            if sample_weights is not None:
                grid_search.fit(X_train, Y_train, sample_weight=sample_weights)
            else:
                grid_search.fit(X_train, Y_train)
                
            train_time = time.time() - start_time
            
            best_model = grid_search.best_estimator_
            best_params = grid_search.best_params_
            
            y_pred = best_model.predict(X_test)
            print(Y_test.shape, y_pred.shape)
            metrics = self.calculate_metrics(Y_test, y_pred)
            metrics['algorithm'] = algo_name
            metrics['train_time'] = train_time
            metrics['best_params'] = best_params
            
            results.append((algo_name, best_model, metrics))
            
            # check type of model to be used get best one
            if self.prediction_type.lower() == "classification":
                if metrics.get('accuracy', 0) > self.best_score:
                    self.best_score = metrics.get('accuracy', 0)
                    self.best_model = best_model
                    self.best_params = best_params
            else:
                if metrics.get('mse', float('inf')) < self.best_score:
                    self.best_score = metrics.get('mse', float('inf'))
                    self.best_model = best_model
                    self.best_params = best_params
        
        return results
    
    def calculate_metrics(self, y_true, y_pred):
        metrics = {}
        
        if self.prediction_type.lower() == "classification":
            metrics['accuracy'] = accuracy_score(y_true, y_pred)
            metrics['precision'] = precision_score(y_true, y_pred, average='weighted', zero_division=0)
            metrics['recall'] = recall_score(y_true, y_pred, average='weighted', zero_division=0)
            metrics['f1'] = f1_score(y_true, y_pred, average='weighted', zero_division=0)
            
            try:
                if len(np.unique(y_true)) == 2:
                    metrics['auc'] = roc_auc_score(y_true, y_pred)
            except:
                metrics['auc'] = 0
        else:
            metrics['mse'] = mean_squared_error(y_true, y_pred)
            metrics['rmse'] = np.sqrt(metrics['mse'])
            metrics['r2'] = r2_score(y_true, y_pred)
        
        return metrics
    
    def predict(self, X_new):
        # predcit from the best
        return self.best_model.predict(X_new)
    
    def get_feature_importance(self):
        if self.best_model is None:
            return None
        
        if hasattr(self.best_model, 'feature_importances_'):
            return self.best_model.feature_importances_
        elif hasattr(self.best_model, 'coef_'):
            return self.best_model.coef_
        else:
            return None
    
def run_pipeline(X_train, X_test, Y_train, Y_test, dic):
    model_pipe = model_pipeline(dic)
    results = model_pipe.train_and_evaluate(X_train, X_test, Y_train, Y_test)
    
    print("\n----- Results Summary -----")
    for algo_name, model, metrics in results:
        print(f"\nAlgorithm: {algo_name}")
        for metric_name, metric_value in metrics.items():
            if metric_name not in ['algorithm', 'train_time', 'best_params']:
                print(f"  {metric_name}: {metric_value}")
        print(f"  Training time: {metrics['train_time']:.2f} seconds")
    
    print(f"\nBest Model: {type(model_pipe.best_model).__name__}")
    print(f"Best Parameters: {model_pipe.best_params}")
    
    if model_pipe.prediction_type.lower() == "classification":
        print(f"Best Accuracy: {model_pipe.best_score:.4f}")
    else:
        print(f"Best MSE: {model_pipe.best_score:.4f}")
    return model_pipe

In [68]:
X_test.shape, Y_test.shape

((90, 10), (45,))

In [69]:
X_train, X_test = k  
Y_train = u[1][2]  
Y_test = u[1][3]   

run_pipeline(X_train, X_test, Y_train, Y_test, dic)

Training RandomForestRegressor...
RandomForestRegressor()
Fitting 6 folds for each of 12 candidates, totalling 72 fits
(45,) (45,)

----- Results Summary -----

Algorithm: RandomForestRegressor
  mse: 0.367543522456339
  rmse: 0.6062536783033476
  r2: -3.914007876496017
  Training time: 1.78 seconds

Best Model: RandomForestRegressor
Best Parameters: {'max_depth': 20, 'min_samples_leaf': 5, 'n_estimators': 20}
Best MSE: 0.3675
